In [ ]:
import pandas as pd
import os, glob
from owlready2 import *

import owlready2
print(owlready2.VERSION) ## Bugs when > 0.40 but works file at owlready2==0.40 

#```Merging ontologies:
#https://stackoverflow.com/questions/75102102/merge-two-ontologies-using-owlready
#```

from datetime import datetime

def NOW():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    return "Current Time = "+ str(current_time)

NOW()

def debug(onto):
    print("\n# Overview\n")
    for k in onto.classes():
        if len(k.instances()):
            print("* ","_"+str(k).split(".")[-1]+"_"," -- ",len(k.instances()), "instances.")

# Destroys empty

In [2]:
onto = get_ontology("../kg/pbn_t3_5.owl").load()
debug(onto)
# Destroy empty classes
for k in onto.classes():
    if not len(k.instances()):
        destroy_entity(k)
# Destroy relations
for k in onto.properties():
    N = len([x for x in k.get_relations()])
    if not N:
        destroy_entity(k)
# Finetune world reasoning
#with onto:
#    sync_reasoner_pellet(infer_property_values = True, infer_data_property_values = True)
# Closing world
#close_world(onto) 
# Saving the KG
onto.save(file = "../kg/new/pbn_t3_5_a.owl")
onto.destroy()
del onto



# Overview

*  _PBNThing_  --  19210 instances.
*  _BenefReturn_  --  1606 instances.
*  _Benef_  --  1409 instances.
*  _Article_  --  376 instances.
*  _Risk_  --  3418 instances.
*  _ISO_Scale_  --  5 instances.
*  _RiskHealth_  --  14 instances.
*  _RiskType_  --  22 instances.
*  _Stakeholder_  --  2640 instances.
*  _Stakeholder_Type_  --  3 instances.
*  _Technology_  --  3915 instances.
*  _ISO_Impact_  --  24 instances.
*  _ISO_Purpose_  --  18 instances.
*  _Mitigation_  --  5295 instances.
*  _BP_Enabler_  --  33 instances.
*  _BP_Transmission_  --  12 instances.
*  _Blueprint_  --  24 instances.
*  _BP_Scale_  --  3 instances.
*  _BP_Phase_  --  4 instances.
*  _BP_Permanent_  --  4 instances.
*  _BP_Type_  --  2 instances.
*  _BP_Intervention_  --  40 instances.
*  _BP_Category_  --  3 instances.
*  _BP_Theme_  --  10 instances.
*  _PBNCategory_  --  330 instances.


# Part B

In [3]:
ontoB = get_ontology("../kg/new/pbn_t3_5_a.owl").load()
for k in ontoB.classes():
    I = k.instances()
    if len(I):
        K = str(k).split(".")[-1]
        #print(K)
        if K.startswith("Blueprint"):
            for i in I:
                destroy_entity(i)
            destroy_entity(k)
            
ontoB.save(file = "../kg/new/pbn_t3_5_noBP.owl")
debug(ontoB)
ontoB.destroy()


# Overview

*  _PBNThing_  --  19186 instances.
*  _BenefReturn_  --  1606 instances.
*  _Benef_  --  1409 instances.
*  _Article_  --  376 instances.
*  _Risk_  --  3418 instances.
*  _ISO_Scale_  --  5 instances.
*  _RiskHealth_  --  14 instances.
*  _RiskType_  --  22 instances.
*  _Stakeholder_  --  2640 instances.
*  _Stakeholder_Type_  --  3 instances.
*  _Technology_  --  3915 instances.
*  _ISO_Impact_  --  24 instances.
*  _ISO_Purpose_  --  18 instances.
*  _Mitigation_  --  5295 instances.
*  _BP_Enabler_  --  33 instances.
*  _BP_Transmission_  --  12 instances.
*  _BP_Scale_  --  3 instances.
*  _BP_Phase_  --  4 instances.
*  _BP_Permanent_  --  4 instances.
*  _BP_Type_  --  2 instances.
*  _BP_Intervention_  --  40 instances.
*  _BP_Category_  --  3 instances.
*  _BP_Theme_  --  10 instances.
*  _PBNCategory_  --  330 instances.


# Part C

In [4]:
ontoC = get_ontology("../kg/new/pbn_t3_5_a.owl").load()
debug(ontoC)
for k in ontoC.classes():
    I = k.instances()
    if len(I):
        K = str(k).split(".")[-1]
        if not( (K.startswith("Blueprint") or K.startswith("BP_") ) ):
            for i in I:
                destroy_entity(i)
            destroy_entity(k)

debug(ontoC)
ontoC.save(file = "../kg/new/pbn_t3_5_BP.owl")
ontoC.destroy()


# Overview

*  _PBNThing_  --  19210 instances.
*  _BenefReturn_  --  1606 instances.
*  _Benef_  --  1409 instances.
*  _Article_  --  376 instances.
*  _Risk_  --  3418 instances.
*  _ISO_Scale_  --  5 instances.
*  _RiskHealth_  --  14 instances.
*  _RiskType_  --  22 instances.
*  _Stakeholder_  --  2640 instances.
*  _Stakeholder_Type_  --  3 instances.
*  _Technology_  --  3915 instances.
*  _ISO_Impact_  --  24 instances.
*  _ISO_Purpose_  --  18 instances.
*  _Mitigation_  --  5295 instances.
*  _BP_Enabler_  --  33 instances.
*  _BP_Transmission_  --  12 instances.
*  _BP_Scale_  --  3 instances.
*  _BP_Phase_  --  4 instances.
*  _BP_Permanent_  --  4 instances.
*  _BP_Type_  --  2 instances.
*  _BP_Intervention_  --  40 instances.
*  _BP_Category_  --  3 instances.
*  _BP_Theme_  --  10 instances.
*  _PBNCategory_  --  330 instances.
*  _Blueprint_  --  24 instances.

# Overview



# Checks

In [5]:
ontoCheck = get_ontology("../kg/new/pbn_t3_5_BP.owl").load()
debug(ontoCheck)
ontoCheck.destroy()


# Overview



# Checks

In [ ]:
onto = get_ontology("../kg/pbn_t3_5.owl").load()
debug(onto)
# Destroy empty classes
for k in onto.classes():
    if not len(k.instances()):
        destroy_entity(k)
# Destroy relations
for k in onto.properties():
    N = len([x for x in k.get_relations()])
    if not N:
        destroy_entity(k)
# Finetune world reasoning
with onto:
    sync_reasoner_pellet(infer_property_values = True, infer_data_property_values = True)
# Closing world
print("Start",NOW())
close_world(onto) 
print("Stop",NOW())
# Saving the KG
onto.save(file = "../kg/new/pbn_t3_5_a_closed.owl")
onto.destroy()
del onto



# Overview

*  _BenefReturn_  --  1606 instances.
*  _Benef_  --  1409 instances.
*  _Article_  --  376 instances.
*  _Risk_  --  3418 instances.
*  _ISO_Scale_  --  5 instances.
*  _RiskHealth_  --  14 instances.
*  _RiskType_  --  22 instances.
*  _Stakeholder_  --  2640 instances.
*  _Stakeholder_Type_  --  3 instances.
*  _Technology_  --  3915 instances.
*  _ISO_Impact_  --  24 instances.
*  _ISO_Purpose_  --  18 instances.
*  _Mitigation_  --  5295 instances.
*  _BP_Enabler_  --  33 instances.
*  _BP_Transmission_  --  12 instances.
*  _BP_Scale_  --  3 instances.
*  _BP_Phase_  --  4 instances.
*  _BP_Permanent_  --  4 instances.
*  _BP_Type_  --  2 instances.
*  _BP_Intervention_  --  40 instances.
*  _BP_Category_  --  3 instances.
*  _BP_Theme_  --  10 instances.
*  _PBNCategory_  --  330 instances.
*  _Blueprint_  --  24 instances.
*  _PBNThing_  --  19210 instances.


* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /home/kelu/.local/lib/python3.10/site-packages/owlready2/pellet/aterm-java-1.6.jar:/home/kelu/.local/lib/python3.10/site-packages/owlready2/pellet/jgrapht-jdk1.5.jar:/home/kelu/.local/lib/python3.10/site-packages/owlready2/pellet/httpcore-4.2.2.jar:/home/kelu/.local/lib/python3.10/site-packages/owlready2/pellet/antlr-runtime-3.2.jar:/home/kelu/.local/lib/python3.10/site-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/home/kelu/.local/lib/python3.10/site-packages/owlready2/pellet/log4j-1.2-api-2.19.0.jar:/home/kelu/.local/lib/python3.10/site-packages/owlready2/pellet/log4j-api-2.19.0.jar:/home/kelu/.local/lib/python3.10/site-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/home/kelu/.local/lib/python3.10/site-packages/owlready2/pellet/xml-apis-1.4.01.jar:/home/kelu/.local/lib/python3.10/site-packages/owlready2/pellet/jena-arq-2.10.0.jar:/home/kelu/.local/lib/python3.10/site-packages/owlready2/pellet/jena-core-2.10.0.jar

* Owlready * Adding relation pbn_t3_5.PBN__Article_248 has_ArticleLLMSummary The coronavirus disease 2019 (COVID-19) outbreak started in China and spread quickly. To stop the spread, China took different measures like travel restrictions to limit people\'s movement. Scientists wanted to understand how these measures affected the spread of COVID-19 in different cities in China. They used data on people\'s movements and case information to study this. They found that before the restrictions, many cases in other cities had traveled from Wuhan, where the outbreak started. But after the restrictions, the number of cases from Wuhan decreased and the virus spread less. This means that the control measures in China helped to reduce the spread of COVID-19.
* Owlready * Adding relation pbn_t3_5.PBN__Article_273 has_ArticleLLMShortSummary The study aimed to compare the literature on occupant health assessment in buildings with professional opinions and understand the impact of the COVID-19 pandem